In [1]:
import pandas as pd
import zipfile
from openpyxl import Workbook

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
# Read data from the 'sub' and 'num' tabs for fraud
fraud_sub_data = pd.read_csv("gdrive/My Drive/Colab Notebooks/SEC_Data/fraud_sub.csv")
fraud_num_data = pd.read_csv("gdrive/My Drive/Colab Notebooks/SEC_Data/fraud_num.csv")


In [4]:
# Read data from the 'sub' and 'num' tabs
sub_data = pd.read_csv("gdrive/My Drive/Colab Notebooks/SEC_Data/sub.csv")
num_data = pd.read_csv("gdrive/My Drive/Colab Notebooks/SEC_Data/num.csv")

In [5]:
# concat non-fraud and fraud data
sub_data = pd.concat([sub_data, fraud_sub_data])
num_data = pd.concat([num_data, fraud_num_data])

In [6]:
# Create a new DataFrame by grouping and summing values for duplicate combinations of 'adsh' and 'tag'
new_df = num_data.groupby(['adsh', 'tag'], as_index=False)['value'].sum()
new_df.head()

,adsh,tag,value
0,0000002178-22-000089,AccountsPayableCurrent,385347000.0
1,0000002178-22-000089,AccountsPayableRelatedPartiesCurrent,20000.0
2,0000002178-22-000089,AccountsReceivableNetCurrent,336579000.0
3,0000002178-22-000089,AccountsReceivableRelatedPartiesCurrent,7000.0
4,0000002178-22-000089,AccruedAutomobileAndWorkersCompensationClaimsC...,8301000.0


In [7]:
# Merge 'sub' and 'num' dataframes based on the 'adsh' column
merged_data = pd.merge(sub_data, new_df[['adsh', 'tag', 'value']], on='adsh', how='left')

# Pivot the 'tag' column into separate columns with 'value' as the values
#pivoted_data = merged_data.pivot(index='adsh', columns='tag', values='value').reset_index()

# Write the resulting data to a new Excel file
#output_file = "output_file.xlsx"
#pivoted_data.to_excel(output_file, index=False, engine='openpyxl')

#print(f"Output file '{output_file}' created successfully.")
#

In [8]:
merged_data.shape

(1164400, 39)

In [9]:
merged_data.head()

,Unnamed: 0,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,...,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks,tag,value
0,0,0000014707-22-000072,14707,CALERES INC,3140.0,US,MO,ST LOUIS,63105,8300 MARYLAND AVE,...,Q3,20221206,2022-12-06 14:50:00.0,0,1,cal-20221029x10q_htm.xml,1,NaN,AccountsPayableTradeCurrent,963258000.0
1,0,0000014707-22-000072,14707,CALERES INC,3140.0,US,MO,ST LOUIS,63105,8300 MARYLAND AVE,...,Q3,20221206,2022-12-06 14:50:00.0,0,1,cal-20221029x10q_htm.xml,1,NaN,AccountsReceivableAllowanceForCreditLossWriteO...,3024000.0
2,0,0000014707-22-000072,14707,CALERES INC,3140.0,US,MO,ST LOUIS,63105,8300 MARYLAND AVE,...,Q3,20221206,2022-12-06 14:50:00.0,0,1,cal-20221029x10q_htm.xml,1,NaN,AccountsReceivableNetCurrent,445495000.0
3,0,0000014707-22-000072,14707,CALERES INC,3140.0,US,MO,ST LOUIS,63105,8300 MARYLAND AVE,...,Q3,20221206,2022-12-06 14:50:00.0,0,1,cal-20221029x10q_htm.xml,1,NaN,AccruedIncomeTaxesCurrent,76099000.0
4,0,0000014707-22-000072,14707,CALERES INC,3140.0,US,MO,ST LOUIS,63105,8300 MARYLAND AVE,...,Q3,20221206,2022-12-06 14:50:00.0,0,1,cal-20221029x10q_htm.xml,1,NaN,AccruedIncomeTaxesNoncurrent,12714000.0


In [10]:
nan_count = merged_data["value"].isna().sum()
nan_count

16558

In [11]:
merged_data_new = merged_data.dropna(subset=['value'])
merged_data_new.shape

(1147842, 39)

In [12]:
merged_data_new = merged_data_new[["adsh", "name", "period", "filed", "tag", "value"]]
merged_data_new.head()

,adsh,name,period,filed,tag,value
0,0000014707-22-000072,CALERES INC,20221031.0,20221206,AccountsPayableTradeCurrent,963258000.0
1,0000014707-22-000072,CALERES INC,20221031.0,20221206,AccountsReceivableAllowanceForCreditLossWriteO...,3024000.0
2,0000014707-22-000072,CALERES INC,20221031.0,20221206,AccountsReceivableNetCurrent,445495000.0
3,0000014707-22-000072,CALERES INC,20221031.0,20221206,AccruedIncomeTaxesCurrent,76099000.0
4,0000014707-22-000072,CALERES INC,20221031.0,20221206,AccruedIncomeTaxesNoncurrent,12714000.0


In [13]:
pivoted_data = merged_data_new.pivot(index='adsh', columns='tag', values='value').reset_index()
pivoted_data.head()

tag,adsh,A155SeniorNotesDue2026,A2.30seniornotesdue2024,A2.95seniornotesdue2029,A2020LaborDayWildfireEarningsTestReserve,A2020SpecialAdvanceOfferingMaximumBalance,A2021NoteFaceAmount,A2021NoteInterestRateStatedPercentage,A2021SeniorNotes,A2022NoteFaceAmount,...,warrantLiabilityExpense,warrants,warrantsToPurchaseSharesOfCommonStock,weightedAveragePricePercentage,workingCapital,workingCapitalDeficit,workingCapitalLoan,workingCapitalLoans,workingCapitalSurplus,workingsCapital
0,0000002178-22-000089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000002488-22-000170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000002969-22-000054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000003499-22-000027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000003570-22-000106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
pivoted_data.shape

(7429, 150851)

In [15]:
nan_count = pivoted_data.isnull().sum()
cols = nan_count[nan_count > 6000]
print(cols)

tag
A155SeniorNotesDue2026                       7428
A2.30seniornotesdue2024                      7428
A2.95seniornotesdue2029                      7428
A2020LaborDayWildfireEarningsTestReserve     7428
A2020SpecialAdvanceOfferingMaximumBalance    7428
                                             ... 
workingCapitalDeficit                        7422
workingCapitalLoan                           7426
workingCapitalLoans                          7428
workingCapitalSurplus                        7428
workingsCapital                              7428
Length: 150709, dtype: int64


In [16]:
cols.index[0]

'A155SeniorNotesDue2026'

In [17]:
pivoted_data_new = pivoted_data.drop(columns=cols.index, axis=1)
pivoted_data_new.shape

(7429, 142)

In [18]:
#pivoted_data_new.to_csv("gdrive/My Drive/Colab Notebooks/SEC_Data/2022q4_secdata.csv")
pivoted_data_new.to_csv("gdrive/My Drive/Colab Notebooks/SEC_Data/2022_secdata_WFL.csv")

In [19]:
#pivoted_data_new = pivoted_data.dropna(axis=1, thresh=1500)
#pivoted_data_new.shape

In [20]:
#pivoted_data_new.head()